In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
import skimage.io
import skimage.transform
import skimage.filters
import skimage.morphology
import skimage.restoration


import numpy
import torch
import scipy
import os

In [6]:
train_data_dir = '../data/train/'
test_data_dir = '../data/test/'

In [7]:
print(len(os.listdir(train_data_dir)), '<- train images')
print(len(os.listdir(test_data_dir)), '<- test images')

670 <- train images
65 <- test images


In [ ]:
# Compute joint mask, contours, centers and ellipsoids
center_percentage = 0.01

for image_name in os.listdir(train_data_dir):
    print(image_name, '<- image name')
    image_path = os.path.join(train_data_dir, image_name)
    masks_path = os.path.join(image_path, 'masks')
    
    contours = None
    centers = None
    ellipsoids = None
    center_locations = None
    
    for mask in os.listdir(masks_path):
        if mask[0] != '.':
            one_mask_path = os.path.join(masks_path, mask)
            
            mask = skimage.io.imread(one_mask_path)
            #if mask.shape[0] < mask.shape[1]:
            #    mask = mask.T
            
            # Computing centers
            XX = numpy.arange(mask.shape[0]).reshape(-1, 1) * numpy.ones(mask.shape)
            YY = numpy.arange(mask.shape[1]).reshape(1, -1) * numpy.ones(mask.shape)
            
            #XX, YY = numpy.meshgrid(numpy.arange(mask.shape[0]), 
            #                        numpy.arange(mask.shape[1]))
            
#             print(mask.shape)
#             print(XX.shape)
#             print(YY.shape)
            x_center = int((XX * mask).sum() / mask.sum())
            y_center = int((YY * mask).sum() / mask.sum())
            
            volume = mask.sum()
            
            if centers is None:
                centers = numpy.zeros_like(mask)
            
            centers[x_center, y_center] = volume
            
            # Computing ellipsoids
            x_coords = XX[mask > 0].flatten().reshape([-1, 1])
            y_coords = YY[mask > 0].flatten().reshape([-1, 1])
            
            coords = numpy.concatenate([x_coords, y_coords], axis=1)
            
            cov = numpy.cov(coords.T)
            
            if ellipsoids is None:
                ellipsoids = numpy.zeros((4,) + mask.shape)
                
            ellipsoids[:, x_center, y_center] = cov.flatten()
            
            mask = skimage.filters.rank.gradient(mask, skimage.morphology.disk(1))
            
            XX = XX - x_center
            YY = YY - y_center
            
            if center_locations is None:
                center_locations = numpy.zeros_like(mask)
            
            coords = numpy.concatenate(
                [XX.reshape((1, XX.size)), YY.reshape((1, YY.size))],
                axis=0)

            dist = (coords * cov.dot(coords)).sum(axis=0).reshape(XX.shape)
            center_locations[dist <= 1.0] = 1.0
            
            
            
            
            
            #for x_index in range(XX.shape[0]):
            #    for y_index in range(YY.shape[1]):
            #        vec = numpy.zeros([2, 1])
            #        vec[0] = XX[x_index, y_index]
            #        vec[1] = YY[x_index, y_index]
            #        
            #        res = vec.T.dot(cov.dot(vec))
            #        if res < 0.01:
            #            center_locations[x_index, y_index] = 1.0
        
            if contours is None:
                contours = mask
            else:
                contours += mask
    
    contours = (contours > 1.0e-2).astype(float)
#     print(contours.shape)
    
    numpy.save(os.path.join(masks_path, '.contours'), contours)
    numpy.save(os.path.join(masks_path, '.centers'), centers)
    numpy.save(os.path.join(masks_path, '.ellipsoids'), ellipsoids)
    numpy.save(os.path.join(masks_path, '.center_locations'), center_locations)
#     break
    
#     print(image_name)

df33b11184427e05c8a450f921586685975fe975f57315e686a0f26fddb93db1 <- image name
aa58ba4512955771b4f9b459cb4e6a8adb71d11cd6cae662ec2df31d688a5fe0 <- image name
9e4f8ec60a0d622a02c0e16eedcc0101f88ddefbcec2383946c4572b57a1e43a <- image name
e66f25e175abab08ecb4e5f6859db64a211e0ddffb262d7e727b9d9bd4aad2d2 <- image name
876423522bdec1602917b94163a21e05fc7b692045219b7bc96cdaf638c33c25 <- image name
3b3f516ebc9a16cff287a5ffd3a1861a345a6d38bedbba74f1c0b0e0eac62afd <- image name
1c8b905c9519061d6d091e702b45274f4485c80dcf7fb1491e6b2723f5002180 <- image name
40bcdad218ac5f0885fc247d88fcad9f729f55c81c79d241a8f1559b6d8c0574 <- image name
dd54adb80393de7769b9853c0aa2ee9b240905d0e99c59d4ccd99401f327aa05 <- image name
b4d902d42c93dea77b541456f8d905f35eeb24fc3a5b0b15b5678d78e0aabe0c <- image name
7b38c9173ebe69b4c6ba7e703c0c27f39305d9b2910f46405993d2ea7a963b80 <- image name
33a5b0ff232b425796ee6a9dd5b516ff9aad54ca723b4ec490bf5cd9b2e2a731 <- image name
bfe8ef193a68a0a86a5e4ae1ddc27bda3f9ffe170494395be403

53ad09e4348767bece0165884bf40c10b72ae18444e3f414a850442f02385efc <- image name
a6e81120d1cb9f71f8a25f90a5d56c4b714a642fc496a705e38921fd90a3f69c <- image name
ebc18868864ad075548cc1784f4f9a237bb98335f9645ee727dac8332a3e3716 <- image name
431b9b0c520a28375b5a0c18d0a5039dd62cbca7c4a0bcc25af3b763d4a81bec <- image name
b1e3aeb0c56261c17eb71c747d116057b8da7e8c8a6845bdc01b2b3ee2299229 <- image name
319b6cb8b0d24b38db5e3c6fbb13b062e2766d9af5ff9bccb8f439ac0d870e52 <- image name
3441821ebea04face181c9e2f4d0d09727c764827ac51b9e7fbadbebabeab225 <- image name
08151b19806eebd58e5acec7e138dbfbb1761f41a1ab9620466584ecc7d5fada <- image name
aa83f5b4fca02ae43a6b9456ab42707b0beabc6e7c5c4e66c0d2572fb80f3615 <- image name
02903040e19ddf92f452907644ad3822918f54af41dd85e5a3fe3e1b6d6f9339 <- image name
76faaed50ed6ea6814ac36199964b86fb09ba7f41a6f213bceaa80d625adc2e1 <- image name
cdab367b30db47061df837c1ae9fa875d6057614f797332d37d3513517d6c694 <- image name
c0152b1a260e71f9823d17f4fbb4bf7020d5dce62b4a12b3099c

In [32]:
print('Done')

Done


In [57]:
def browse_images(data_path, masks=True, weight=4.0):
#     n = len(digits.images)
    images = os.listdir(data_path)
    
    def view_image(image_index):
        print(images[image_index])
        image_path = os.path.join(data_path, images[image_index])
        content_path = os.path.join(image_path, 
                                    'images/' + images[image_index] + '.png')
        image = skimage.io.imread(content_path).astype(float)
#         image = skimage.restoration.denoise_tv_chambolle(image, weight=weight)
            
#         if masks is not None:
            

        if masks:
            masks_dir = os.path.join(image_path, 
                             'masks')

#             print(os.listdir(masks_dir))
            contours = numpy.load(os.path.join(masks_dir, '.contours.npy'))
            centers = numpy.load(os.path.join(masks_dir, '.centers.npy'))
        
            image[:, :, 1] += contours * 255
            image[:, :, 0] += centers * 255
        
        image = image.clip(0, 255) / 256
        
#         print(image.shape)
#         print(image.max(), image.min())
        
        plt.figure(figsize=(10.0, 10.0))
        plt.imshow(image)
        plt.show()
    interact(view_image, 
             image_index=widgets.IntSlider(min=0,max=len(images) - 1,step=1,value=0))

In [58]:
browse_images(train_data_dir)

interactive(children=(IntSlider(value=0, description='image_index', max=669), Output()), _dom_classes=('widget-interact',))

In [54]:
browse_images(train_data_dir, masks=False)

interactive(children=(IntSlider(value=0, description='image_index', max=669), Output()), _dom_classes=('widget-interact',))

In [ ]:
# Плохо размеченные картинки:
# 7b38c9173ebe69b4c6ba7e703c0c27f39305d9b2910f46405993d2ea7a963b80
